In [ ]:
from src.utils import *
from src.llm_gen import *
from src.dataset_llm_templates import *
from src.data_loader import *

import pandas as pd
from copy import deepcopy
import pickle
import time
from sklearn.model_selection import train_test_split


api_details = {
     "api_base": "https://api.together.xyz/v1",
     "api_version": "2023-07-01-preview",
     "api_key": "7bea21d91b0bccc19c60fc797e37b33e5f16ae0c31ea3e52ebe1294a8227ff30"
}

model_short_name = 'mixtral'
model = "mistralai/Mixtral-8x7B-Instruct-v0.1"


for seed in [0,1,2,3,4,5,6,7,8,9,10]:
    for ns in [10, 20, 50, 100]: 
        for dataset in ["compas"]:
            try:
                # sleep between runs from a rate limit perspective
                time.sleep(120)
                n_synthetic=1000
                n_processes = 5

                df_feat, df_label, df = get_data(dataset=dataset, seed=seed)


                X_train, X_remain, y_train, y_remain = sample_and_split(df_feat, df_label, ns=ns, seed=seed)

                X_val, X_test, y_val, y_test = train_test_split(
                    X_remain, y_remain, test_size=0.5, random_state=seed
                )


                X_train_orig = deepcopy(X_train)
                y_train_orig = deepcopy(y_train)

                results = {}
                results['Original'] = {"X": X_train_orig, 'y': y_train_orig}
                results['Oracle'] = {"X": X_val, 'y': y_val}
                results['Test'] = {"X": X_test, 'y': y_test}


                prompt, generator_template, format_instructions, example_df = langchain_templates(X_train_orig, y_train_orig, dataset=dataset)

                retries = 4  # Max retries you want to attempt

                while retries > 0:
                    try:

                        if len(example_df)>20:
                            ic_samples=20
                        else:
                            ic_samples=len(example_df)
                        
                        print(f'Running {dataset}, {seed}, {model} --- {n_processes}')
                        df_llm = llm_gen(prompt, generator_template, format_instructions, example_df, 
                                        n_samples=n_synthetic,
                                        temperature=0.9,
                                        max_tokens=1000, model=model, 
                                        n_processes=n_processes,
                                        ic_samples=ic_samples)
                        
                        break  # if successful, break out of the loop
                    except Exception as e:
                        time.sleep(120)
                        print(f"Error: {e}. Retrying with reduced n_processes...")
                        n_processes = int(n_processes/2)
                        retries -= 1
                        if n_processes < 1:
                            print("Error: Minimum n_processes reached. Exiting...")
                            break
                try:
                    tmp_df = df_llm.astype(example_df.dtypes)
                    df_llm = tmp_df
                except:
                    pass

                ylabel_map = {'covid': "is_dead",
                            "adult": "salary",
                            "compas": "y",
                            "drug": "y",
                            "bio": "y",
                            "higgs": "y",
                            "seer": "mortCancer",
                            "cutract":"mortCancer",
                            "maggic": "death_all",
                            "support": "death",
                            }
                
                ylabel =  ylabel_map[dataset]

                X_train_llm = df_llm.drop(columns=[ylabel])
                y_train_llm = df_llm[ylabel]

                results['llm'] = {"X": X_train_llm, 'y': y_train_llm, 'df': df}


                with open(f'../save_dfs/pipeline_llm_{dataset}_{n_synthetic}_{model_short_name}_{ns}_{seed}.pickle', 'wb') as f:
                        pickle.dump(results, f)

            except Exception as e:
                import traceback
                print(traceback.format_exc())
                print(e)
                continue